In [8]:
!apt-get install nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java libatk-wrapper-java-jni
  libbabeltrace1 libcub-dev libcublas11 libcublaslt11 libcudart11.0 libcufft10 libcufftw10
  libcuinj64-11.5 libcupti-dev libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libfontenc1 libgail-common libgail18
  libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglvnd-core-dev libglvnd-dev libglx-dev
  libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11 libnppidei11
  libnppif11 libnppig11 libnppim11 libnppist11 libnppisu11 libnppitc11 libnpps11 libnvblas11
  libnvidia-compute-495 libnvidia-compute-510 libnvidia-compute-535 libnvidia-ml-dev libnvjpeg11
  libnvrtc-builtins11.5 libnvrtc11.2 libnvtoolsext1 libnvvm4 libopengl-dev libr

In [9]:
!nvidia-smi

Fri Apr 26 12:25:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [26]:
%%writefile ass1.cu
#include<iostream>
#include<cuda_runtime.h>
#include<cstdlib>
using namespace std;

__global__ void multi(int* a, int* b, int* c,int size){
  int row=blockIdx.y * blockDim.y + threadIdx.y;
  int col=blockIdx.x * blockDim.x + threadIdx.x;
  if(row<size && col<size){
    int sum=0;
    for(int i=0;i<size;i++){
      sum+=a[row*size+i] * b[i*size +col];
    }
    c[row*size+col]=sum;
  }
}
void initialize(int* matrix, int size){
  for(int i=0;i<size*size;i++){
    matrix[i]=rand()%10;
  }
}
void print(int* matrix,int size){
  for(int row=0;row<size;row++){
    for(int col=0;col<size;col++){
      cout<<matrix[row*size+col]<<" ";
    }
    cout<<endl;
  }
  cout<<endl;
}
int main(){
  int* a,* b,* c;
  int n=2;
  int matrixSize=n*n;
  size_t matrixBytes=matrixSize* sizeof(int);
  a=new int[matrixSize];
  b=new int[matrixSize];
  c=new int[matrixSize];

  initialize(a,n);
  initialize(b,n);

  cout<<"matrix A:";
  print(a, n);
  cout<<"matrix B:";
  print(b,n);

  int* x,* y,* z;
  cudaMalloc(&x,matrixBytes);
  cudaMalloc(&y,matrixBytes);
  cudaMalloc(&z,matrixBytes);
  cudaMemcpy(x, a, matrixBytes, cudaMemcpyHostToDevice);
  cudaMemcpy(y, b, matrixBytes, cudaMemcpyHostToDevice);
  int THREADS=2;
  int BLOCKS=n/THREADS;
  dim3 threads(THREADS, THREADS);
  dim3 blocks(BLOCKS, BLOCKS);
  multi<<<blocks,threads>>>(x,y,z,n);
  cudaMemcpy(c,z,matrixBytes,cudaMemcpyDeviceToHost);
  cout<<"Multiplication: ";
  print(c,n);

  delete[] a;
  delete[] b;
  delete[] c;

  cudaFree(x);
  cudaFree(y);
  cudaFree(z);
  return 0;


}

Overwriting ass1.cu


In [27]:
!nvcc -o ass1 ass1.cu

In [28]:
!./ass1

matrix A:3 6 
7 5 

matrix B:3 5 
6 2 

Multiplication: 45 27 
51 45 

